# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [ ]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
from panel.interact import interact
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [ ]:
# Read the Mapbox API key
load_dotenv("../../../resources/mapbox_keys.env")
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)
type(map_box_api)

# Import Data

In [ ]:
# Import the necessary CSVs to Pandas DataFrames
sfo_data_filepath = Path("data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(sfo_data_filepath, index_col="year")

loc_data_filepath = Path("data/neighborhoods_coordinates.csv")
loc_data = pd.read_csv(loc_data_filepath).set_index('Neighborhood')

sfo_data.head()

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [ ]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    
    sfo_data_grouped_year = sfo_data.groupby(['year'])
    sfo_data_mean_units = sfo_data_grouped_year['housing_units'].mean().reset_index()
    
    bottom_lim = int(sfo_data_mean_units['housing_units'].min() - sfo_data_mean_units['housing_units'].std())
    top_lim = int(sfo_data_mean_units['housing_units'].max() + sfo_data_mean_units['housing_units'].std())
    
    '''
    Bug Report:
    Plots had to be re-done in px.plot's (indead of pd.plot used in rental_analysis) due to a bug with pd and px, where:
       - when calling different methods that return different pd.plots to be displayed in px.Row or px.Column:
           - it seems like the pd.plots are treated as pyplot.axes within pd plotting which uses pyplot
           - this in turn, maps all of the plots onto 1 subplot of the pd.plot, overlaying them
           - in the end, if plot backgrouds are transparent:
               - one overlayed plot is displayed on all plot instanced within the px.Row or px.Column
           - or, if plot backgrouds are NOT transparent:
               - it seems as the last plot is displayed on all plot instanced within the px.Row or px.Column
    '''
    
#     return sfo_data_mean_units.plot.bar(title="SF Mean Number of Housing Units Per Year", 
#                                        ylim=(bottom_lim, top_lim)).get_figure()


    housing_units_per_year_plot = px.bar(
        sfo_data_mean_units,
        x="year",
        y="housing_units",
        title="Housing Units Per Year",
        labels={
            "year" : 'Year',
            "housing_units" : 'Units, n'
            },
        range_y=[bottom_lim, top_lim])
    
    return housing_units_per_year_plot

def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    sfo_data_grouped_year = sfo_data.groupby(['year'])
    gross_rent = sfo_data_grouped_year['gross_rent'].mean().reset_index()

    '''
    Bug Report:
    Plots had to be re-done in px.plot's (indead of pd.plot used in rental_analysis) due to a bug with pd and px, where:
       - when calling different methods that return different pd.plots to be displayed in px.Row or px.Column:
           - it seems like the pd.plots are treated as pyplot.axes within pd plotting which uses pyplot
           - this in turn, maps all of the plots onto 1 subplot of the pd.plot, overlaying them
           - in the end, if plot backgrouds are transparent:
               - one overlayed plot is displayed on all plot instanced within the px.Row or px.Column
           - or, if plot backgrouds are NOT transparent:
               - it seems as the last plot is displayed on all plot instanced within the px.Row or px.Column
    '''
    
#     gross_rent.plot(color='red', 
#                            title='Average Gross Rent by Year', 
#                            ylabel = 'Price per Sqft',
#                            xlabel = 'Year')

    gross_rent_plot = px.line(
        gross_rent,
        x="year",
        y="gross_rent",
        title="Average Gross Rent by Year",
        labels={
            "year" : 'Year',
            "gross_rent" : 'Gross Rent, $'
            }
        )
    
    return gross_rent_plot

def average_sales_price():
    """Average Sales Price Per Year."""
    
    sale_price_sqr_foot = sfo_data.groupby(['year'])['sale_price_sqr_foot'].mean().reset_index()
    
    '''
    Bug Report:
    Plots had to be re-done in px.plot's (indead of pd.plot used in rental_analysis) due to a bug with pd and px, where:
       - when calling different methods that return different pd.plots to be displayed in px.Row or px.Column:
           - it seems like the pd.plots are treated as pyplot.axes within pd plotting which uses pyplot
           - this in turn, maps all of the plots onto 1 subplot of the pd.plot, overlaying them
           - in the end, if plot backgrouds are transparent:
               - one overlayed plot is displayed on all plot instanced within the px.Row or px.Column
           - or, if plot backgrouds are NOT transparent:
               - it seems as the last plot is displayed on all plot instanced within the px.Row or px.Column
    '''
    
#     return sale_price_sqr_foot.plot(color='purple', 
#                                     title='Average Sale Price by Year',
#                                     ylabel = 'Price per Sqft',
#                                     xlabel = 'Year')

    sale_price_sqr_foot_plot = px.line(
        sale_price_sqr_foot,
        x="year",
        y="sale_price_sqr_foot",
        title="Average Sale Price by Year",
        labels={
            "year" : 'Year',
            "sale_price_sqr_foot" : 'Sale Price per Square Foot, $'
            })
    
    return sale_price_sqr_foot_plot

def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    sfo_data_year_hood = sfo_data.groupby(['year','neighborhood']).mean().reset_index()
    
    return sfo_data_year_hood.hvplot.line(x='year', 
                                          y='sale_price_sqr_foot',
                                          title='Average Prices by Neighborhood',
                                          groupby='neighborhood', 
                                          ylabel='Avg. Sale Price per Square Foot')



def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    
    sfo_data_max10 = sfo_data.groupby(['neighborhood']).mean().nlargest(10,'sale_price_sqr_foot')
    df_expensive_neighborhoods_per_year = sfo_data[sfo_data["neighborhood"].isin(sfo_data_max10.reset_index()["neighborhood"])]

    return df_expensive_neighborhoods_per_year.hvplot.bar(x='year',
                                         y=['sale_price_sqr_foot','gross_rent'],
                                         title='Most Expensive Neighborhoods in San Francisco',
                                         ylabel='Num. Housing Units',
                                         xlabel='Year',
                                         groupby='neighborhood', 
                                         rot=90, color=['red','blue'],
                                         height=500)




def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""

    sfo_data_max10 = sfo_data.groupby(['neighborhood']).mean().nlargest(10,'sale_price_sqr_foot')['sale_price_sqr_foot'].reset_index()   
    
    '''
    Bug Report:
    Plots had to be re-done in px.plot's (indead of pd.plot used in rental_analysis) due to a bug with pd and px, where:
       - when calling different methods that return different pd.plots to be displayed in px.Row or px.Column:
           - it seems like the pd.plots are treated as pyplot.axes within pd plotting which uses pyplot
           - this in turn, maps all of the plots onto 1 subplot of the pd.plot, overlaying them
           - in the end, if plot backgrouds are transparent:
               - one overlayed plot is displayed on all plot instanced within the px.Row or px.Column
           - or, if plot backgrouds are NOT transparent:
               - it seems as the last plot is displayed on all plot instanced within the px.Row or px.Column
    '''
        
#     return sfo_data_max10['sale_price_sqr_foot'].plot.bar(rot=90,
#                                                           title='Top 10 Expensive Neightborhoods in SFO',
#                                                           ylabel='Avg. Sale Price Per Square Foot')

    sfo_data_max10_plot = px.bar(
        sfo_data_max10,
        x="neighborhood",
        y="sale_price_sqr_foot",
        title="Top 10 Most Expensive Neighborhoods",
        labels={
            "neighborhood" : 'Neighborhood',
            "sale_price_sqr_foot" : 'Sale Price per Square Foot, $'
            })
    
    return sfo_data_max10_plot



def parallel_coordinates():
    """Parallel Coordinates Plot."""
    
    sfo_data_max10 = sfo_data.groupby(['neighborhood']).mean().nlargest(10,'sale_price_sqr_foot')
    df_expensive_neighborhoods_per_year = sfo_data[sfo_data["neighborhood"].isin(sfo_data_max10.reset_index()["neighborhood"])]

    return px.parallel_coordinates(df_expensive_neighborhoods_per_year.reset_index(), 
                                 color='year', 
                                 title = 'San Francisco Housing Market',
                                 labels = {
                                     'year' : 'Year',
                                     'neighborhood' : 'Neighborhood',
                                     'sale_price_sqr_foot' : '$ per Sq. Foot',
                                     'housing_units' : 'Housing Units',
                                     'gross_rent' : 'Gross Rent'
                                 })



def parallel_categories():
    """Parallel Categories Plot."""
        
    sfo_data_max10 = sfo_data.groupby(['neighborhood']).mean().nlargest(10,'sale_price_sqr_foot')
    df_expensive_neighborhoods_per_year = sfo_data[sfo_data["neighborhood"].isin(sfo_data_max10.reset_index()["neighborhood"])]

    return px.parallel_categories(df_expensive_neighborhoods_per_year.reset_index(), 
                                 color='year', 
                                 title = 'Most Expensive Neighborhoods',
                                 labels = {
                                     'year' : 'Year',
                                     'neighborhood' : 'Neighborhood',
                                     'sale_price_sqr_foot' : '$ per Sq. Foot',
                                     'housing_units' : 'Housing Units',
                                     'gross_rent' : 'Gross Rent'
                                 })


def neighborhood_map():
    """Neighborhood Map."""

    loc_data_filepath = Path("data/neighborhoods_coordinates.csv")
    loc_data = pd.read_csv(loc_data_filepath).set_index('Neighborhood')
    mean_data = sfo_data.groupby(['neighborhood']).mean()
    mean_data_loc = pd.concat([mean_data, loc_data], axis='columns', join='inner')

    mapbox_plot = px.scatter_mapbox(
        mean_data_loc,
        lat="Lat",
        lon="Lon",
        size="sale_price_sqr_foot",
        color="gross_rent",
        zoom = 10)
    
    return mapbox_plot


def sunburst():
    """Sunburst Plot."""
    
    sfo_data_max10 = sfo_data.groupby(['neighborhood']).mean().nlargest(10,'sale_price_sqr_foot')
    df_expensive_neighborhoods_per_year = sfo_data[sfo_data["neighborhood"].isin(sfo_data_max10.reset_index()["neighborhood"])]

    sunburst_plot = px.sunburst(
        df_expensive_neighborhoods_per_year.reset_index(),
        path=['year','neighborhood'], 
        values='sale_price_sqr_foot')
    
    return sunburst_plot

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [ ]:
# Create a Title for the Dashboard
dashboard_title = pn.Column(
    '# San Francisco Real Estate Analysis',
    background='WhiteSmoke')


# Create Tab Contents
welcome_col = pn.Column("## Welcome", 
                        "This dashboard presents a visual analysis of the San Francisco real estate market",
                        neighborhood_map(),
                       background='WhiteSmoke')


yearly_analysis_row = pn.Row(housing_units_per_year(), average_gross_rent(), average_sales_price(), width_policy='max')


neighbor_analysis_col = pn.Column(
    pn.Column("### Average Price Analysis", average_price_by_neighborhood()),
    pn.Column("### Most Expensive Home Analysis",
        pn.panel(top_most_expensive_neighborhoods(), width_policy='max' ),
        pn.panel(most_expensive_neighborhoods_rent_sales(), width_policy='fit'),
        background='WhiteSmoke'
         )
    )


    
parallel_plots_col = pn.Column("## Parallel Plots Analysis",
                               parallel_coordinates(),
                                parallel_categories(),
                               background='WhiteSmoke')

sunburst_col = pn.Column("## Sunburst Plot Analysis", 
                         sunburst(),
                         background='WhiteSmoke')

    
# Create a tab layout for the dashboard
real_estate_dashboard = pn.Tabs(
    ("Welcome", welcome_col),
    ("Yearly Market Analysis", yearly_analysis_row),
    ("Neighborhood Analysis", neighbor_analysis_col),
    ("Parallel Plots Analysis", parallel_plots_col),
    ("Sunburst Plot Analysis", sunburst_col)   
)

## Serve the Panel Dashboard

In [ ]:
# Serve the# dashboard
real_estate_dashboard.servable()

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
# housing_units_per_year()

In [ ]:
# average_gross_rent()

In [ ]:
# average_sales_price()

In [ ]:
# average_price_by_neighborhood()

In [ ]:
# top_most_expensive_neighborhoods()

In [ ]:
# most_expensive_neighborhoods_rent_sales()

In [ ]:
# neighborhood_map().show()

In [ ]:
# parallel_categories()

In [ ]:
# parallel_coordinates()

In [ ]:
# sunburst()

In [ ]:
# list(sfo_data['neighborhood'])